In [22]:
import pandas as pd
df = pd.read_csv('data.csv')
df['category'] = df['category'].map({'Best category': 1, 'Worst category': 0})
df.head()

,name,Criterion 1,Criterion 2,Criterion 3,Criterion 4,category
0,Alternative 1,0.374540,0.796543,0.950714,0.183435,1
1,Alternative 2,0.731994,0.779691,0.598659,0.596850,1
2,Alternative 3,0.156019,0.445833,0.155995,0.099975,0
3,Alternative 4,0.058084,0.459249,0.866176,0.333709,1
4,Alternative 5,0.601115,0.142867,0.708073,0.650889,1


In [23]:
thresholds = [
    [0.255905151],
    [0.676961303],
    [0.0551739074],
    [0.324553937]
]

for i in range(1, 5):
  criterion_name = f"Criterion {i}"
  df[criterion_name] = (df[criterion_name] > thresholds[i-1][0]).astype(int)

df.head()


,name,Criterion 1,Criterion 2,Criterion 3,Criterion 4,category
0,Alternative 1,1,1,1,0,1
1,Alternative 2,1,1,1,1,1
2,Alternative 3,0,0,1,0,0
3,Alternative 4,0,0,1,1,1
4,Alternative 5,1,0,1,1,1


In [24]:
df.head()

# Check for NaNs in the DataFrame
print(df.isnull().values.any())

False


In [34]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt
from mlxtend.plotting import plot_decision_regions
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA


X = df[['Criterion 1', 'Criterion 2', 'Criterion 3', 'Criterion 4']].values
y = df['category'].values
from sklearn.svm import LinearSVC

svm_classifier = LinearSVC(fit_intercept=False)

svm_classifier.fit(X, y)

y_pred = svm_classifier.predict(X)

print("Accuracy:", accuracy_score(y, y_pred))
print(classification_report(y, y_pred))


Accuracy: 0.97
              precision    recall  f1-score   support

           0       1.00      0.88      0.93        24
           1       0.96      1.00      0.98        76

    accuracy                           0.97       100
   macro avg       0.98      0.94      0.96       100
weighted avg       0.97      0.97      0.97       100



In [31]:
import numpy as np

w = svm_classifier.coef_[0]
w

array([-0.76539385,  1.60032197, -0.09843702,  1.71935793])

In [33]:
df['w*X'] = (df[['Criterion 1', 'Criterion 2', 'Criterion 3', 'Criterion 4']].values @ w)
mn = min(df['w*X'].loc[~(df['w*X'] < 0)])
w = w/mn
print(w)
df['w*X'] /= mn
df['w*X >= 1'] = (df['w*X'] >= 1).astype(int)
df['check'] = (df['w*X >= 1'] == df['category']).astype(int)
print(df['check'].value_counts())
df.head()
#criterion_weights: [0.0723807141, 0.186306968, 0.868274033, 0.538130879]


[-1.03924387  2.17290065 -0.13365677  2.33452645]
check
1    97
0     3
Name: count, dtype: int64


,name,Criterion 1,Criterion 2,Criterion 3,Criterion 4,category,w*X,w*X >= 1,check
0,Alternative 1,1,1,1,0,1,1.000000,1,1
1,Alternative 2,1,1,1,1,1,3.334526,1,1
2,Alternative 3,0,0,1,0,0,-0.133657,0,1
3,Alternative 4,0,0,1,1,1,2.200870,1,1
4,Alternative 5,1,0,1,1,1,1.161626,1,1
